Took help from the follwing repository: https://github.com/udacity/deep-learning-v2-pytorch/blob/master/sentiment-rnn/Sentiment_RNN_Solution.ipynb. It helped in building the model. 

In [1]:
import csv
from string import punctuation
from collections import Counter
import numpy as np
import torch

In [3]:
rows = []
labels = []
titles = []
posts = []

with open('train.csv') as train_file:
  reader = csv.reader(train_file)
  for row in reader:
    rows.append(row)
  for i in range(1, len(rows)):
    labels.append(rows[i][2])
    posts.append(rows[i][1])
    titles.append(rows[i][0])

In [4]:
print(titles[0])

netflix the family has been an amazing watch do you think the rssvhp has been what family at c street has been to the bjp and their annual meet up is what the national prayer breakfast held in washington is all about


In [5]:
t = open('titles.txt', 'w')
p = open('posts.txt', 'w')

In [6]:
l = open('labels.txt', 'w')

In [7]:
len(titles)

36547

In [8]:
len(posts)

36547

In [9]:
len(labels)

36547

In [10]:
for i in range(len(titles)):
  if i == len(titles) - 1:
    t.write(titles[i])
  else:
    t.write(titles[i] + '\n')

In [63]:
min(labels)

tensor(0)

In [11]:
for i in range(len(posts)):
  if i == len(posts) - 1:
    p.write(posts[i])
  else:
    p.write(posts[i] + '\n')

In [12]:
for i in range(len(labels)):
  if i == len(labels) - 1:
    l.write(labels[i])
  else:
    l.write(labels[i] + '\n')

In [13]:
with open('titles.txt') as t:
  title = t.read()
with open('posts.txt') as p:
  post = p.read()
with open('labels.txt') as l:
  label = l.read()

In [14]:
# get rid of punctuation
title = title.lower() # lowercase, standardize
all_text = ''.join([c for c in title if c not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words_t = all_text.split()

In [15]:
# get rid of punctuation
post = post.lower() # lowercase, standardize
all_text = ''.join([c for c in post if c not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words_p = all_text.split()

In [16]:
words_t[:30]

['netflix',
 'the',
 'family',
 'has',
 'been',
 'an',
 'amazing',
 'watch',
 'do',
 'you',
 'think',
 'the',
 'rssvhp',
 'has',
 'been',
 'what',
 'family',
 'at',
 'c',
 'street',
 'has',
 'been',
 'to',
 'the',
 'bjp',
 'and',
 'their',
 'annual',
 'meet',
 'up']

In [17]:
## Build a dictionary that maps words to integers
counts = Counter(words_p)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
title_ints = []
for titl in reviews_split:
    title_ints.append([vocab_to_int[word] for word in titl.split()])

In [18]:
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()
print('Tokenized review: \n', title_ints[:1])

Unique words:  91284

Tokenized review: 
 [[31457, 117, 1028, 1, 159, 8, 36, 5, 17213, 403, 6, 22880, 3188, 519, 55, 14, 881, 4481, 2, 48, 6, 1, 105, 1952, 42, 7, 11237, 7421]]


In [19]:
labels_split = label.split('\n')
labels_split = labels_split[:36548]
encoded_labels = np.array([int(label_value) for label_value in labels_split])

In [20]:
len(encoded_labels)

36547

In [21]:
for i in range(len(encoded_labels)):
  encoded_labels[i] = int(encoded_labels[i])

In [22]:
type(encoded_labels[0])

numpy.int64

In [23]:
# Outliers
title_lens = Counter([len(x) for x in title_ints])
print("Zero-length titles: {}".format(title_lens[0]))
print("Maximum title length: {}".format(max(title_lens)))

Zero-length reviews: 505
Maximum review length: 4379


In [24]:
print('Number of titles before removing outliers: ', len(title_ints))

non_zero_idx = [ii for ii, review in enumerate(title_ints) if len(review) != 0]

reviews_ints = [title_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of titles after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  36547
Number of reviews after removing outliers:  36042


In [25]:
def pad_features(reviews_ints, seq_length):
    # Pads or truncates the titles to a fixed length given by seq_length
    
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [26]:
seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

print(features[:30,:10])

[[   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [ 423  368    5  289  806  572  216  320  909   17]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   3   50  109    5  147  558   26    5 1901  470]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0

In [28]:
# Splitting the data
train_x, train_y = features, encoded_labels
train_data = torch.utils.data.TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))

batch_size = 50

# Define the train_loader
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)

In [29]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ..., 16094,     7,  1548],
        [    0,     0,     0,  ...,     0,     0, 19359],
        [    0,     0,     0,  ...,   174,    41,  2849],
        ...,
        [    5,   117,   487,  ...,     2,    35,   367],
        [  410,    42,     3,  ...,    99,  1172,  1416],
        [    0,     0,     0,  ...,     3,   116, 38441]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([14,  7,  0,  0,  0, 10,  0,  0,  0,  0, 10,  7,  0, 10,  7,  0, 14,  0,
        10, 10,  0,  0,  0,  0,  0,  1,  0,  3,  0,  0,  7,  0,  0, 10,  0,  0,
         0,  0,  7,  7,  4,  0,  0,  9,  7,  9,  9, 12,  0,  0])


In [78]:
# Checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [101]:
import torch.nn as nn

class RNN(nn.Module):

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(RNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        #self.softmax = nn.Softmax(dim=1)
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        #print(x.shape)
        embeds = self.embedding(x)
        #print(x.shape)
        lstm_out, hidden = self.lstm(embeds, hidden)
        #print(lstm_out.shape)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        #print(lstm_out.shape)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        #print(out.shape)
        out = self.fc(out)
        #print(out.shape)
        # softmax function
        #out = self.softmax(out)
        
        # reshape to be batch_size first
        out = out.view(batch_size, -1)
        #print(out.shape)
        out = out[:, -1] # get last batch of labels
        #print(out.shape)
        
        # return last sigmoid output and hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [102]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 15
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

RNN(
  (embedding): Embedding(91285, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=15, bias=True)
  (softmax): Softmax(dim=1)
)


In [103]:
# loss and optimization functions
lr=0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [105]:
# Training

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip = 5 # gradient clipping to prevent exploding gradient problem

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        #print(torch.argmax(labels))
        #print(labels)
        #print(inputs)
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Create new varaiables to prevent back propogating through the entire history
        h = tuple([each.data for each in h])

        # Zero the accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)
        output = output.unsqueeze(dim=0)
        print(output.shape)

        labels = torch.argmax(labels)
        # calculate the loss and perform backprop
        loss = criterion(output, labels)
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
    print('Epoch: {}'.format(e+1),
          'Training Loss: {}'.format(loss.item()))

tensor(41)
THIS IS THE BATCH_SIZE: 50
torch.Size([50, 200])
torch.Size([50, 200])
torch.Size([50, 200, 256])
torch.Size([10000, 256])
torch.Size([10000, 256])
torch.Size([10000, 15])
torch.Size([50, 3000])
torch.Size([50])
torch.Size([1, 50])
This is output: tensor([[0.0707, 0.0718, 0.0704, 0.0690, 0.0735, 0.0668, 0.0704, 0.0702, 0.0733,
         0.0705, 0.0734, 0.0683, 0.0688, 0.0735, 0.0679, 0.0705, 0.0733, 0.0689,
         0.0711, 0.0676, 0.0732, 0.0683, 0.0707, 0.0714, 0.0696, 0.0686, 0.0697,
         0.0691, 0.0686, 0.0713, 0.0697, 0.0707, 0.0747, 0.0694, 0.0695, 0.0703,
         0.0715, 0.0741, 0.0680, 0.0688, 0.0707, 0.0706, 0.0685, 0.0663, 0.0696,
         0.0718, 0.0708, 0.0703, 0.0703, 0.0678]],
       grad_fn=<UnsqueezeBackward0>)


TypeError: ignored